# Check Fringe Benefits
This is slightly harder than pensions as we have to combine GIC statewide and miscellanous fringe benefit spending from the agencies themselves. 
Good news is that it's exact same methodology for each state agency so I don't have to check multiple state agencies.
Should I look for other statewide benefits? There don't seem to be any on the scale of GIC

## Set up environment

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import os
import sys
import numpy as np
from sodapy import Socrata

In [3]:
sys.path.insert(0, os.path.abspath("../.."))

In [4]:
app_token = "2Qa1WiG8G4kj1vGVd2noK7zP0"
client = Socrata("cthru.data.socrata.com", app_token)
client.timeout = 40

In [5]:
from Initialize_Agencies_3Ver import get_agencies
yr = list(range(2016,2020))
agencies = get_agencies(yr)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/Users/alexanderweinstein/Documents/Harris/Summer2020/Carceral_Budgeting/Exploratory/pensions/State_Pensions_2Ver.py:139: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

contributions by year


budget_fiscal_year
2016   1695121996.790
2017   1774642712.500
2018   1825313115.460
2019   1909137852.920
2020   1991497887.030
Name: amount, dtype: float64

/Users/alexanderweinstein/Documents/Harris/Summer2020/Carceral_Budgeting/Exploratory/Initialize_Agencies_3Ver.py:5: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  from Agency_Classes_2Ver import StateAgency, CPCS, MBTA, BostonPD, ChelseaPD, ReverePD, WinthropPD
/Users/alexanderweinstein/Documents/Harris/Summer2020/Carceral_Budgeting/Exploratory/Agency_Classes_2Ver.py:184: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  self.get_expenditures_by_year()
/Users/alexanderweinstein/Documents/Harris/Summer2020/Carceral_Budgeting/Exploratory/Agency_Classes_2Ver.py:178: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  self.get_final_costs()
/Users/alexanderweinstein/Documents/Harris/Summer2020/Carceral_Budgeting/Exploratory/Agency_Classes_2Ver.py:184: DtypeWarning: Columns (15,16) have mixed types.Specify dtype option on import or set 

## Get Statewide Health Insurance

In [6]:
GIC = pd.DataFrame(client.get("pegc-naaa",
                                                         where = "department = 'GROUP INSURANCE COMMISSION (GIC)'" + 
                                                         "AND budget_fiscal_year >=2016", 
                                                         limit =999999))

In [7]:
GIC["budget_fiscal_year"] = GIC["budget_fiscal_year"].astype(int)
GIC["amount"] = GIC["amount"].astype(float)


99.6%

In [8]:
GIC_no_federal = GIC[GIC["appropriation_type"].str.contains("(?i)federal")==False]
statewide_health_insurance = GIC[GIC["object_code"] =="(D06) EMPLOYEE HEALTH & LIFE INSURANCE"]
SHI_by_year = statewide_health_insurance.groupby("budget_fiscal_year").sum()["amount"]

In [9]:
SHI_by_year

budget_fiscal_year
2016   2676540536.150
2017   2737028846.550
2018   2702163794.500
2019   2717337093.050
2020   2745379043.540
2021   1003313342.220
Name: amount, dtype: float64

## Check State Police
We need: payroll fraction for statewide and expenditures to get insurance-related

In [10]:
sys.path.insert(0, os.path.abspath("../../fringe"))
from Statewide_Fringe import Total_Statewide_Payroll

In [11]:
statewide_payroll = Total_Statewide_Payroll(client)

In [12]:
state_police_exp = pd.DataFrame(client.get("pegc-naaa",
                                                         where = "department = 'DEPARTMENT OF STATE POLICE (POL)'" + 
                                                         "AND budget_fiscal_year >=2016", 
                                                         limit =999999))

ConnectionError: HTTPSConnectionPool(host='cthru.data.socrata.com', port=443): Read timed out.

In [ ]:
state_police_exp["amount"] = state_police_exp["amount"].astype(float)
state_police_exp["budget_fiscal_year"] = state_police_exp["budget_fiscal_year"].astype(int)

Remove federal, intragovernmental, payroll, capital

In [ ]:
state_police_exp = state_police_exp[state_police_exp["appropriation_type"].str.contains("INTRAGOVERNMENTAL")==False]
state_police_exp = state_police_exp[state_police_exp["appropriation_type"].str.contains("(?i)federal")==False]
state_police_exp = state_police_exp[state_police_exp["appropriation_type"].str.contains("(?i)capital")==False]
state_police_exp = state_police_exp[state_police_exp["vendor"].str.contains("(?i)payroll")==False]



In [ ]:
sp_fringe_nonhidden = state_police_exp[state_police_exp["object_class"] == "(DD) PENSION & INSURANCE RELATED EX"]

In [ ]:
sp_nonhidden_by_year = sp_fringe_nonhidden.groupby("budget_fiscal_year").sum()["amount"]
sp_nonhidden_by_year

In [ ]:
state_police = agencies["State_Police"]
sp_hidden_by_year = SHI_by_year*(state_police.payroll_by_year/statewide_payroll)

In [ ]:
assert state_police.fringe.equals((sp_hidden_by_year + sp_nonhidden_by_year).loc[list(range(2016,2020))])

Ok great that all looks good. Thankfully exactly one methodology for all state agencies

## Check Boston
Methodology is that we use health insurance, worker's comp as the two statewide fringe benefits to take fraction of.
Are there any line-items in budget to be counted as fringe benefits? Yes, worker's comp medical

Check 2017, 2018

In [ ]:
citywide_health_insurance_2017 = 205281017
citywide_workers_comp_2017  = 1478695
PD_budget_workers_comp_2017 = 132969

## Check Chelsea

## Check Revere

## Check Winthrop